In [18]:
import os
import datetime
import selenium.webdriver
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [19]:
codelist=['BLR','BOM','MAA','DEL']
numdays = 2

In [20]:
base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
#date_list

In [21]:
date_strings=[]
for i in range(len(date_list)):
    date_str_i=date_list[i].strftime("%d/%m/%Y")
    date_strings.append(date_str_i)
    #print(date_str_i)
    
date_strings[0:5]

['22/10/2019', '23/10/2019']

In [22]:
url1="https://www.makemytrip.com/flight/search?tripType=O&itinerary="
url4="&paxType=A-1_C-0_I-0&cabinClass=E"
for i in range(len(codelist)):
    codelist_noi= list(set(codelist) - set([codelist[i]]))
    for j in range(len(codelist_noi)):
        place=codelist[i]+"-"+codelist_noi[j]+"-"
        url2=url1+place
        #print(url2)
        for k in range(len(date_strings)):
            url3=url2+date_strings[k]+url4
            print(url3)
            
            try:
                driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe') # Chrome driver is being used.
                print ("Requesting URL: " + url3)

                driver.get(url3)  			 # URL requested in browser.
                print ("Webpage found ...")

                element_xpath = '//*[@id="left-side--wrapper"]/div[2]' # First box with relevant flight data.

                # Wait until the first box with relevant flight data appears on Screen
                element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

                # Scroll the page till bottom to get full data available in the DOM.
                print ("Scrolling document upto bottom ...")
                for j in range(1, 100):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
                body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

                print("Closing Chrome ...") # No more usage needed.
                driver.quit() 				# Browser Closed.

                print("Getting data from DOM ...")
                soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup

                # Extract the required tags 
                spanFlightName = soupBody.findAll("span", {"class": "airways-name"}) 			# Tags with Flight Name
                pFlightCode = soupBody.findAll("p", {"class": "fli-code"})				# Tags with Flight Code
                divDeptTime = soupBody.findAll("div", {"class": "dept-time"})				# Tags with Departure Time
                pDeptCity = soupBody.findAll("p", {"class": "dept-city"})				# Tags with Departure City
                pFlightDuration = soupBody.findAll("p", {"class": "fli-duration"})			# Tags with Flight Duration
                pArrivalTime = soupBody.findAll("p", {"class": "reaching-time append_bottom3"}) 	# Tags with Arrival Time
                pArrivalCity = soupBody.findAll("p", {"class": "arrival-city"})				# Tags with Arrival City
                spanFlightCost = soupBody.findAll("span", {"class": "actual-price"})			# Tags with Flight Cost


                # Data Headers
                flightsData = [["flight_name", "flight_code", "departure_time", "departure_city", "flight_duration", "arrival_time", "arrival_city", "flight_cost"]]

                # Extracting data from tags and appending to main database flightsData
                for j in range(0, len(spanFlightName)):
                    flightsData.append([spanFlightName[j].text, pFlightCode[j].text, divDeptTime[j].text, pDeptCity[j].text, pFlightDuration[j].text, pArrivalTime[j].text, pArrivalCity[j].text, spanFlightCost[j].text])

                # Output File for FlightsData. This file will have the data in comma separated form.
                outputFile = "FlightsData_" + place+date_strings[k].replace("/","-")+".csv"

                # Publishing Data to File
                print("Writing flight data to file: "+ outputFile + " ...")
                with open(outputFile, 'w', newline='', encoding="utf-8") as spfile:
                    csv_writer = csv.writer(spfile)
                    csv_writer.writerows(flightsData)
                    print ("Data Extracted and Saved to File. ")

            except Exception as e:
                print (str(e))

https://www.makemytrip.com/flight/search?tripType=O&itinerary=BLR-MAA-22/10/2019&paxType=A-1_C-0_I-0&cabinClass=E
Requesting URL: https://www.makemytrip.com/flight/search?tripType=O&itinerary=BLR-MAA-22/10/2019&paxType=A-1_C-0_I-0&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
Writing flight data to file: FlightsData_BLR-MAA-22-10-2019.csv ...
Data Extracted and Saved to File. 
https://www.makemytrip.com/flight/search?tripType=O&itinerary=BLR-MAA-23/10/2019&paxType=A-1_C-0_I-0&cabinClass=E
Requesting URL: https://www.makemytrip.com/flight/search?tripType=O&itinerary=BLR-MAA-23/10/2019&paxType=A-1_C-0_I-0&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
Writing flight data to file: FlightsData_BLR-MAA-23-10-2019.csv ...
Data Extracted and Saved to File. 
https://www.makemytrip.com/flight/search?tripType=O&itinerary=BLR-BOM-22/10/2019&paxType=A-1_C-0_I-0&cabinClas